In [2]:
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)

/opt/homebrew/Cellar/jupyterlab/3.3.2/libexec/bin/python3.9
3.9.13 (main, May 24 2022, 21:13:51) 
[Clang 13.1.6 (clang-1316.0.21.2)]
sys.version_info(major=3, minor=9, micro=13, releaselevel='final', serial=0)


In [1]:
import sys
sys.path.append("/Users/hugo.paula/Documents/hugo_doc/mimo_emb_fts/src/")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from embfts.models.MimoNonStationaryFtsPca import MimoNonStationaryFtsPca
from pyFTS.models.nonstationary import nsfts
from pyFTS.benchmarks import Measures
from pyFTS.benchmarks import Measures
import matplotlib.pyplot as plt
from pyFTS.common import Util
import datetime
import statistics
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score
import math


np.seterr(divide='ignore', invalid='ignore')

/opt/homebrew/lib/python3.9/site-packages/pyFTS/common/transformations/trend.py:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  from pandas import datetime


ModuleNotFoundError: No module named 'tensorflow'

In [2]:
def sample_first_prows(data, perc):
    return data.head(int(len(data)*(perc)))

In [3]:
def cal_nrmse(rmse, y):
    x = max(y)-min(y)
    return (rmse/x)

In [4]:
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame) 
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

In [5]:
# convert series to supervised learning
def series_to_supervised_mimo(data, n_in, n_out, dropnan=True):
	"""
	Frame a time series as a supervised learning dataset.
	Arguments:
		data: Sequence of observations as a list or NumPy array.
		n_in: Number of lag observations as input (X).
		n_out: Number of observations as output (y).
		dropnan: Boolean whether or not to drop rows with NaN values.
	Returns:
		Pandas DataFrame of series framed for supervised learning.
	"""
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [(df.columns[j]+'(t-%d)' % (i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [(df.columns[j]+'(t)') for j in range(n_vars)]
		else:
			names += [(df.columns[j]+'%d(t+%d)' % (j, i)) for j in range(n_vars)]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [6]:
df = pd.read_csv('/home/hugo/projetos-doutorado/mimo_emb_fts/data/energydata_complete.csv', sep=',')
df = df.drop(labels=['date','rv1', 'rv2'], axis=1)
df = clean_dataset(df)

In [7]:
# frame as supervised learning
#reframed = series_to_supervised_mimo(df, 1, 1)
reframed = df
reframed.head()

,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,...,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint
0,60.0,30.0,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,...,18.2,48.900000,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3
1,60.0,30.0,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,...,18.2,48.863333,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2
2,50.0,30.0,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,...,18.2,48.730000,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1
3,50.0,40.0,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,...,18.1,48.590000,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0
4,60.0,40.0,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,...,18.1,48.590000,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9


In [8]:
cols = reframed.columns
cols

Index(['Appliances', 'lights', 'T1', 'RH_1', 'T2', 'RH_2', 'T3', 'RH_3', 'T4',
       'RH_4', 'T5', 'RH_5', 'T6', 'RH_6', 'T7', 'RH_7', 'T8', 'RH_8', 'T9',
       'RH_9', 'T_out', 'Press_mm_hg', 'RH_out', 'Windspeed', 'Visibility',
       'Tdewpoint'],
      dtype='object')

In [9]:
# df_forecast.rename(columns = {'Appliances(t-1)': 'Appliances(t)', 
#                               'lights(t-1)':'lights(t)', 'T1(t-1)':'T1(t)', 
#                               'RH_1(t-1)':'RH_1(t)', 'T2(t-1)':'T2(t)',
#                               'RH_2(t-1)':'RH_2(t)', 'T3(t-1)':'T3(t)', 
#                               'RH_3(t-1)':'RH_3(t)', 'T4(t-1)':'T4(t)', 
#                               'RH_4(t-1)':'RH_4(t)', 'T5(t-1)':'T5(t)',
#                               'RH_5(t-1)':'RH_5(t)', 'T6(t-1)':'T6(t)', 
#                               'RH_6(t-1)':'RH_6(t)', 'T7(t-1)':'T7(t)', 
#                               'RH_7(t-1)':'RH_7(t)', 'T8(t-1)':'T8(t)',
#                               'RH_8(t-1)':'RH_8(t)', 'T9(t-1)':'T9(t)', 
#                               'RH_9(t-1)':'RH_9(t)', 'T_out(t-1)':'T_out(t)', 
#                               'Press_mm_hg(t-1)':'Press_mm_hg(t)',
#                               'RH_out(t-1)':'RH_out(t)', 'Windspeed(t-1)':'Windspeed(t)', 
#                               'Visibility(t-1)':'Visibility(t)','Tdewpoint(t-1)':'Tdewpoint(t)'}, 
#                                inplace = True)

In [10]:
# df_forecats_coluns = ['Appliances(t)', 'lights(t)', 'T1(t)', 'RH_1(t)', 'T2(t)', 'RH_2(t)',
#        'T3(t)', 'RH_3(t)', 'T4(t)', 'RH_4(t)', 'T5(t)', 'RH_5(t)', 'T6(t)',
#        'RH_6(t)', 'T7(t)', 'RH_7(t)', 'T8(t)', 'RH_8(t)', 'T9(t)', 'RH_9(t)',
#        'T_out(t)', 'Press_mm_hg(t)', 'RH_out(t)', 'Windspeed(t)',
#        'Visibility(t)', 'Tdewpoint(t)']

In [12]:
def sliding_window_pca(data,n_windows,train_size,order,steps_ahead,transformation):

    result = {
         "window": [],
         "rmse": [],
         "mape": [],
         "mae": [],
         "r2": [],
         "smape": [],
         "nrmse": [],
         "variable":[]
    }
    
    final_result = {
         "window": [],
         "rmse": [],
         "mape": [],
         "mae": [],
         "r2": [],
         "smape": [],
         "nrmse": [],
         "variable":[]
    }

    tam = len(data)
    n_windows = n_windows
    windows_length = math.floor(tam / n_windows)
    for ct, ttrain, ttest in Util.sliding_window(data, windows_length, train_size, inc=1):
        if len(ttest) > 0:
            
            print('-' * 20)
            print(f'training window {(ct)}')
            
#             df_train = ttrain.loc[:,'Appliances(t-1)':'Tdewpoint(t-1)']
#             df_test = ttest.loc[:,'Appliances(t-1)':'Tdewpoint(t-1)']
#             df_original = ttest.loc[:,'Appliances(t)':'Tdewpoint(t)']

#             Xtrain = ttrain.loc[:,'Appliances(t-1)':'Tdewpoint(t-1)']
#             ytrain = ttrain.loc[:,'Appliances(t)':'Tdewpoint(t)']
#             Xtest = ttest.loc[:,'Appliances(t-1)':'Tdewpoint(t-1)']
#             ytest = ttest.loc[:,'Appliances(t)':'Tdewpoint(t)']
            
            models, data_train = mimo_kpca_nsfts.run_train(ttrain,transformation)
            forecast, data_test = mimo_kpca_nsfts.run_test(models,ttest,steps_ahead,transformation)

#             models, data_train = mimo_kpca_nsfts.run_train(Xtrain,transformation)
#             forecast, data_test = mimo_kpca_nsfts.run_test(models,Xtest,steps_ahead,transformation)
            
            
#             df_forecats_columns = ['Appliances(t)', 'lights(t)', 'T1(t)', 'RH_1(t)', 'T2(t)', 'RH_2(t)',
#                                    'T3(t)', 'RH_3(t)', 'T4(t)', 'RH_4(t)', 'T5(t)', 'RH_5(t)', 'T6(t)',
#                                    'RH_6(t)', 'T7(t)', 'RH_7(t)', 'T8(t)', 'RH_8(t)', 'T9(t)', 'RH_9(t)',
#                                    'T_out(t)', 'Press_mm_hg(t)', 'RH_out(t)', 'Windspeed(t)',
#                                    'Visibility(t)', 'Tdewpoint(t)']
                                                           
#            columns = list(ytest.columns)
#            df_forecast = pd.DataFrame(forecast,columns=columns)
            #df_forecast = pd.DataFrame(forecast,columns=ttest.columns)
    
            columns = list(ttest.columns)
            df_forecast = pd.DataFrame(forecast,columns=columns)
            
#             print("input")
#             print(df_test.loc[:,'Appliances(t-1)'][1:50])
            
            for col in columns:  
                original = ttest[col].values
                forecast = df_forecast[col].values
                original = original[order:]
                forecast = forecast[:len(forecast)-order]
                #residuals = np.array(original) - np.array(forecast)

                
#                 fig, ax = plt.subplots(nrows=1, ncols=1, figsize=[15, 3])
#                 ax.plot(original, label='Original')
#                 ax.plot(forecast, label='Forecast')
#                 ax.plot(residuals, label='Residuals')
#                 handles, labels = ax.get_legend_handles_labels()
#                 lgd = ax.legend(handles, labels, loc=2, bbox_to_anchor=(1, 1))
#                 plt.show()
                
                #print("[{0: %H:%M:%S}]".format(datetime.datetime.now()) + f" getting statistics for variable: " + col)
                mae = round(mean_absolute_error(original,forecast),3)
                r2 = round(r2_score(original,forecast),3)
                #rmse = mean_squared_error(original,forecast,squared=False)
                rmse = round(Measures.rmse(original,forecast),3)
                mape = round(Measures.mape(original,forecast),3)
                nrmse = round(cal_nrmse(rmse, original),3)
                smape = round(Measures.smape(original,forecast),3)

                result["rmse"].append(rmse)
                result["nrmse"].append(nrmse)
                result["mape"].append(mape)
                result["mae"].append(mae)
                result["r2"].append(r2)
                result["smape"].append(smape)
                result["window"].append(ct)
                result["variable"].append(col)
                
                
        
    measures = pd.DataFrame(result)
    return measures

## Sliding Window: MIMO KPCA-NSFTS

In [13]:
num_components_pca = 3
order = 1
npart = 50
gamma = 0.1
fts_model = nsfts.NonStationaryFTS
memory_window_error = 3
steps_ahead = 1

mimo_kpca_nsfts = MimoNonStationaryFtsPca(num_components_pca, order, npart,
                                            gamma, fts_model, memory_window_error)

In [14]:
kpca_result =  sliding_window_pca(data=reframed,n_windows=30,order=order,train_size=0.75,steps_ahead=1,transformation='KPCA')

--------------------
training window 0
--------------------
training window 657
--------------------
training window 1314
--------------------
training window 1971
--------------------
training window 2628
--------------------
training window 3285
--------------------
training window 3942
--------------------
training window 4599
--------------------
training window 5256
--------------------
training window 5913
--------------------
training window 6570
--------------------
training window 7227
--------------------
training window 7884
--------------------
training window 8541
--------------------
training window 9198
--------------------
training window 9855
--------------------
training window 10512
--------------------
training window 11169
--------------------
training window 11826
--------------------
training window 12483
--------------------
training window 13140
--------------------
training window 13797
--------------------
training window 14454
--------------------
training w

In [16]:
columns = list(reframed.columns)
#columns = list(df_forecats_coluns)


final_result = {
    "variable": [],
    "rmse": [],
    "mae": [],
    "mape": [],
    "r2": [],
    "smape": [],
    "nrmse": [],
    "rmse_std": [],
    "mae_std": [],
    "mape_std": [],
    "r2_std": [],
    "smape_std": [],
    "nrmse_std": []
}

kpca_result.to_csv (r'all_windows_kpca_mo_ensfts_uci_appliances.csv', index = False, header=True)

measures = kpca_result
var = measures.groupby("variable")

for col in columns:
    
    var_agr = var.get_group(col)
           
    rmse = round(statistics.mean(var_agr.loc[:,'rmse']),3)
    mape = round(statistics.mean(var_agr.loc[:,'mape']),3)
    mae = round(statistics.mean(var_agr.loc[:,'mae']),3)
    r2 = round(statistics.mean(var_agr.loc[:,'r2']),3)
    smape = round(statistics.mean(var_agr.loc[:,'smape']),3)
    nrmse = round(statistics.mean(var_agr.loc[:,'nrmse']),3)
    
    rmse_std = round(statistics.stdev(var_agr.loc[:,'rmse']),3)
    mape_std = round(statistics.stdev(var_agr.loc[:,'mape']),3)
    mae_std = round(statistics.stdev(var_agr.loc[:,'mae']),3)
    r2_std = round(statistics.stdev(var_agr.loc[:,'r2']),3)
    smape_std = round(statistics.stdev(var_agr.loc[:,'smape']),3)
    nrmse_std = round(statistics.stdev(var_agr.loc[:,'nrmse']),3)

    final_result["variable"].append(col)
    final_result["rmse"].append(rmse)
    final_result["mape"].append(mape)
    final_result["mae"].append(mae)
    final_result["r2"].append(r2)
    final_result["smape"].append(smape)
    final_result["nrmse"].append(nrmse)
    
    final_result["rmse_std"].append(rmse_std)
    final_result["mape_std"].append(mape_std)
    final_result["mae_std"].append(mae_std)
    final_result["r2_std"].append(r2_std)
    final_result["smape_std"].append(smape_std)
    final_result["nrmse_std"].append(nrmse_std)
        
    #print(f'Results: {(col,rmse,mae,r2)}')
        
        
final_measures_kpca = pd.DataFrame(final_result) 



In [17]:
print("Statistics MIMO KPCA-NSFTS(test): ")
final_measures_kpca

Statistics MIMO KPCA-NSFTS(test): 


,variable,rmse,mae,mape,r2,smape,nrmse,rmse_std,mae_std,mape_std,r2_std,smape_std,nrmse_std
0,Appliances,64.758,29.653,25.947,4.060000e-01,11.631,0.125,29.058,13.149,5.116,2.470000e-01,1.913,0.028
1,lights,4.998,2.489,inf,1.590000e-01,84.215,inf,1.730,1.270,NaN,6.320000e-01,13.084,NaN
2,T1,0.061,0.046,0.212,9.870000e-01,0.106,0.033,0.017,0.014,0.071,8.000000e-03,0.036,0.008
3,RH_1,0.424,0.208,0.506,9.170000e-01,0.254,0.050,0.212,0.085,0.205,5.700000e-02,0.103,0.014
4,T2,0.119,0.087,0.416,9.880000e-01,0.208,0.032,0.062,0.045,0.181,5.000000e-03,0.091,0.007
5,RH_2,0.283,0.190,0.484,9.750000e-01,0.242,0.037,0.129,0.092,0.258,2.100000e-02,0.129,0.011
6,T3,0.074,0.049,0.215,9.700000e-01,0.108,0.041,0.034,0.024,0.096,1.700000e-02,0.048,0.011
7,RH_3,0.203,0.119,0.303,9.580000e-01,0.152,0.044,0.103,0.051,0.129,2.800000e-02,0.064,0.010
8,T4,0.083,0.053,0.258,9.760000e-01,0.129,0.039,0.048,0.030,0.153,1.600000e-02,0.077,0.008
9,RH_4,0.171,0.116,0.302,9.720000e-01,0.151,0.038,0.053,0.036,0.102,2.000000e-02,0.051,0.012


In [18]:
final_measures_kpca.to_csv (r'kpca_mo_ensfts_uci_appliances_with_std.csv', index = False, header=True)

## Sliding Window: MIMO PCA-NSFTS

In [19]:
num_components_pca = 10
order = 1
npart = 50
gamma = 0.1
fts_model = nsfts.NonStationaryFTS
memory_window_error = 3
steps_ahead = 1

mimo_kpca_nsfts = MimoNonStationaryFtsPca(num_components_pca, order, npart,
                                            gamma, fts_model, memory_window_error)

In [20]:
pca_result =  sliding_window_pca(data=reframed,n_windows=30,order=order,train_size=0.75,steps_ahead=1,transformation='PCA')

--------------------
training window 0
--------------------
training window 657
--------------------
training window 1314
--------------------
training window 1971
--------------------
training window 2628
--------------------
training window 3285
--------------------
training window 3942
--------------------
training window 4599
--------------------
training window 5256
--------------------
training window 5913
--------------------
training window 6570
--------------------
training window 7227
--------------------
training window 7884
--------------------
training window 8541
--------------------
training window 9198
--------------------
training window 9855
--------------------
training window 10512
--------------------
training window 11169
--------------------
training window 11826
--------------------
training window 12483
--------------------
training window 13140
--------------------
training window 13797
--------------------
training window 14454
--------------------
training w

In [21]:
#columns = list(df_forecats_coluns)
columns = list(reframed.columns)

final_result = {
    "variable": [],
    "rmse": [],
    "mae": [],
    "mape": [],
    "r2": [],
    "smape": [],
    "nrmse": [],
    "rmse_std": [],
    "mae_std": [],
    "mape_std": [],
    "r2_std": [],
    "smape_std": [],
    "nrmse_std": []
}

pca_result.to_csv (r'all_windows_pca_mo_ensfts_uci_appliances.csv', index = False, header=True)

measures = pca_result
var = measures.groupby("variable")

for col in columns:
    
    var_agr = var.get_group(col)
           
    rmse = round(statistics.mean(var_agr.loc[:,'rmse']),3)
    mape = round(statistics.mean(var_agr.loc[:,'mape']),3)
    mae = round(statistics.mean(var_agr.loc[:,'mae']),3)
    r2 = round(statistics.mean(var_agr.loc[:,'r2']),3)
    smape = round(statistics.mean(var_agr.loc[:,'smape']),3)
    nrmse = round(statistics.mean(var_agr.loc[:,'nrmse']),3)

    rmse_std = round(statistics.stdev(var_agr.loc[:,'rmse']),3)
    mape_std = round(statistics.stdev(var_agr.loc[:,'mape']),3)
    mae_std = round(statistics.stdev(var_agr.loc[:,'mae']),3)
    r2_std = round(statistics.stdev(var_agr.loc[:,'r2']),3)
    smape_std = round(statistics.stdev(var_agr.loc[:,'smape']),3)
    nrmse_std = round(statistics.stdev(var_agr.loc[:,'nrmse']),3)

    final_result["variable"].append(col)
    final_result["rmse"].append(rmse)
    final_result["mape"].append(mape)
    final_result["mae"].append(mae)
    final_result["r2"].append(r2)
    final_result["smape"].append(smape)
    final_result["nrmse"].append(nrmse)
    
    final_result["rmse_std"].append(rmse_std)
    final_result["mape_std"].append(mape_std)
    final_result["mae_std"].append(mae_std)
    final_result["r2_std"].append(r2_std)
    final_result["smape_std"].append(smape_std)
    final_result["nrmse_std"].append(nrmse_std)
        
    #print(f'Results: {(col,rmse,mae,r2)}')
        
        
final_measures_pca = pd.DataFrame(final_result) 

print("Statistics MIMO PCA-NSFTS(test): ")
final_measures_pca

Statistics MIMO PCA-NSFTS(test): 


,variable,rmse,mae,mape,r2,smape,nrmse,rmse_std,mae_std,mape_std,r2_std,smape_std,nrmse_std
0,Appliances,62.155,35.231,37.650,0.453,16.223,0.121,27.913,17.268,12.615,0.220,4.418,0.027
1,lights,4.793,3.082,inf,0.222,83.073,NaN,1.640,1.339,NaN,0.598,13.178,NaN
2,T1,0.101,0.076,0.353,0.968,0.176,0.051,0.049,0.027,0.139,0.020,0.069,0.014
3,RH_1,0.546,0.345,0.850,0.867,0.426,0.066,0.245,0.140,0.350,0.082,0.176,0.015
4,T2,0.191,0.142,0.681,0.971,0.340,0.050,0.105,0.075,0.314,0.013,0.157,0.012
5,RH_2,0.393,0.295,0.752,0.950,0.376,0.054,0.164,0.125,0.364,0.033,0.182,0.016
6,T3,0.117,0.089,0.396,0.932,0.198,0.062,0.058,0.043,0.179,0.028,0.089,0.015
7,RH_3,0.294,0.209,0.535,0.912,0.268,0.066,0.122,0.078,0.204,0.044,0.102,0.015
8,T4,0.132,0.099,0.485,0.946,0.243,0.061,0.079,0.059,0.306,0.023,0.153,0.013
9,RH_4,0.250,0.188,0.489,0.939,0.245,0.055,0.081,0.062,0.180,0.045,0.090,0.018


In [22]:
final_measures_pca.to_csv (r'pca_mo_ensfts_uci_appliances_with_std.csv', index = False, header=True)